yield是内建关键词

In [ ]:
def generate_square(n):
    i = 0
    while i < n:
        yield i * i
        i += 1

result = generate_square(10)
# next(result)  <=> result.__next__() implement 2 method ( protocal: __iter__() __next__() )
print(list(result))
next(result)

In [2]:
# 上面的这段代码会计算0-9的平方并打印出来。

# 那么问题来了，这段代码和下面的这段代码有什么区别呢？

def generate_square(n):
    i = 0
    result = []
    while i < n:
        result.append(i * i)
        i += 1
    return result

result = generate_square(10)
print(result)

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]


这里的关键点是，前一段代码使用了yield关键字。

那么yield是什么呢？要理解yield，还得从容器开始说起。

容器（container）
像列表（list）、集合（set）、序列（tuple）、字典（dict）都是容器。
简单的说，容器是一种把多个元素组织在一起的数据结构，可以逐个迭代获取其中的元素。容器可以用in来判断容器中是否包含某个元素，如

'a' in {'a', 'b', 'c'} # 输出 True
'a' in {'a': 1, 'b': 2} # 输出 True
'a' in set(['a', 'b', 'c']) # 输出 True

大多数的容器都是可迭代对象，可以使用某种方式访问容器中的每一个元素。

迭代器（iterator）
实现了__iter__和__next__方法的对象都称为迭代器。

迭代器是一个有状态的对象，在调用next() 的时候返回下一个值，如果容器中没有更多元素了，则抛出StopIteration异常。

看下面的例子

In [7]:
a = ['a', 'b', 'c']
it = a.__iter__()
print(next(it))
print(next(it))
print(next(it))
print(next(it))

a
b
c


StopIteration: 

In [4]:
#为更好地理解迭代器的内部运行机制，我们再来看一个斐波那契数列的例子

class Fib:
    def __init__(self):
        self.prev = 0
        self.curr = 1

    def __iter__(self):
        return self

    def __next__(self):
        self.curr, self.prev = self.prev + self.curr, self.curr
        return self.curr

fib = Fib()
for i in range(10):
    print(next(fib))

1
2
3
5
8
13
21
34
55
89


只要不断地调用next() 方法，上面的生成器可以生成一个无限长的斐波那契数列。

迭代器是一种Lasy Load的模式，只有在调用时才生成值，没有调用的时候就等待下一次调用。


生成器和yield
生成器其实是一种特殊的迭代器，但是不需要像迭代器一样实现__iter__和__next__方法，只需要使用关键字yield就可以。

我们来实现一个同样的斐波那契数列，但这次使用的是生成器


In [5]:
def fib():
    prev, curr = 0, 1
    while True:
        yield curr
        curr, prev = prev + curr, curr

f = fib()
for i in range(10):
    print(next(f))

1
1
2
3
5
8
13
21
34
55


上面的 fib 函数中没有 return 关键字。
当运行 f = fib() 的时候，它返回的是一个生成器对象。
在调用 fib() 的时候并不会运行 fib 函数中的代码，只有在调用 next() 的时候才会真正运行其中的代码。

两种方式实现的generate_square函数，一个使用了yield关键字，一个使用了列表保存所有的值并返回列表，两者的区别在什么地方？

对于前一种实现方式，使用了生成器，在调用函数的时候不会一次性生成所有的元素，而是在每次调用 next() 才生成一个元素；而后一种方式，在调用函数的时候就生成了所有元素，相比之下，更耗费内存和CPU。



看到这里，大家是不是理解了yield关键字呢？

那么给大家出一个思考题：下面的代码为什么第二次调用next打印None呢？



In [11]:
def echo(n):
    while True:
        n = yield n

g = echo(1)
print(next(g))
print(next(g))

1
None


yield的机制是暂停运行过程输出当前结果并保留状态，状态包括上次终止的位置和终止时的数值。
下一次next()时从上一次终止的地方开始，因为yield之后没有任何语句所以打印结果是None


首先，先把yield看做“return”，这个是直观的，它首先是个return。

In [13]:
def foo():
    print("starting...")
    while True:
        res = yield 4
        print("res:",res)
g = foo()
print(next(g))
print("*"*20)
print(next(g))
# print(g.send(7))

starting...
4
********************
res: None
4


相当于代码单步调试：

1.程序开始执行以后，因为foo函数中有yield关键字，所以foo函数并不会真的执行，而是先得到一个生成器g(相当于一个对象)

2.直到调用next方法，foo函数正式开始执行，先执行foo函数中的print方法，然后进入while循环

3.程序遇到yield关键字，然后把yield想想成return,return了一个4之后，程序停止，并没有执行赋值给res操作，此时next(g)语句执行完成，所以输出的前两行（第一个是while上面的print的结果,第二个是return出的结果）是执行print(next(g))的结果，

4.程序执行print("*"*20)，输出20个*

5.又开始执行下面的print(next(g)),这个时候和上面那个差不多，不过不同的是，这个时候是从刚才那个next程序停止的地方开始执行的，也就是要执行res的赋值操作，这时候要注意，这个时候赋值操作的右边是没有值的（因为刚才那个是return出去了，并没有给赋值操作的左边传参数），所以这个时候res赋值是None,所以接着下面的输出就是res:None,

6.程序会继续在while里执行，又一次碰到yield,这个时候同样return 出4，然后程序停止，print函数输出的4就是这次return出的4.

 
带yield的函数是一个生成器，而不是一个函数了，这个生成器有一个函数就是next函数，next就相当于“下一步”生成哪个数，
这一次的next开始的地方是接着上一次的next停止的地方执行的，所以调用next的时候，生成器并不会从foo函数的开始执行，只是接着上一步停止的地方开始，
然后遇到yield后，return出要生成的数，此步就结束。

In [15]:
8208 == 8**4 + 2**4 + 8**4

True

In [4]:
def fib(x):
    a ,b= 1,1
    for i in range(x):
        yield a
        a ,b = b,a+b
        

list(fib(5))

[1, 1, 2, 3, 5]

In [16]:
def shuixianhua(n):        
    for i in range(1,n):
        exp = 1
        ls = []
        while i // exp > 0 :
            ls.append(i // exp % 10)
            exp *= 10
        p = len(ls)
        if sum([item**p for item in ls])==i:
            yield i
 
list(shuixianhua(1000000))

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 153,
 370,
 371,
 407,
 1634,
 8208,
 9474,
 54748,
 92727,
 93084,
 548834]

In [10]:
def fib(x):
    a,b = 1,1
    for i in range(x):
        yield a # return a, 同时保存当前状态，中断！！！
        a , b = b, a+b

f = fib(10)
list(f)
 


[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]